# New merges for master_pcts
* Losing appeals cases because of an inner join
* These appeals child cases aren't associated with an AIN, but we can grab it from parent case
* But, merging using PARENT_CASE rather than CASE_ID means it's a m:m merge (gross!)
* Figure out if we can expand the merges, switch the order, and create new master_pcts
* Add print statements to check the length of the df

In [1]:
import pandas as pd
import geopandas as gpd
import intake

catalog = intake.open_catalog("../catalogs/*.yml")
bucket_name = 'city-planning-entitlements'

# Import data
cases = pd.read_parquet(f's3://{bucket_name}/data/raw/tCASE.parquet')
app = pd.read_parquet(f's3://{bucket_name}/data/raw/tAPLC.parquet')
geo_info = pd.read_parquet(f's3://{bucket_name}/data/raw/tPROP_GEO_INFO.parquet')
la_prop = pd.read_parquet(f's3://{bucket_name}/data/raw/tLA_PROP.parquet')

In [2]:
# Subset dataframes before merging
keep_col = ['CASE_ID', 'APLC_ID', 'CASE_NBR', 
                'CASE_SEQ_NBR', 'CASE_YR_NBR', 'CASE_ACTION_ID', 
                'CASE_FILE_RCV_DT', 'CASE_FILE_DATE', 'PARNT_CASE_ID']

cases1 = (cases.assign(
    # Grab the year-month from received date
    CASE_FILE_DATE = pd.to_datetime(cases['CASE_FILE_RCV_DT']).dt.to_period('M'),
)[keep_col])

app1 = app[['APLC_ID', 'PROJ_DESC_TXT']]
geo_info1 = geo_info[['CASE_ID', 'PROP_ID']].drop_duplicates()
la_prop1 = la_prop[la_prop.ASSR_PRCL_NBR.notna()][['PROP_ID', 'ASSR_PRCL_NBR']]

# Identify parent cases
cases1['parent_is_null'] = cases1.PARNT_CASE_ID.isna()
cases1['PARENT_CASE'] = cases1.apply(lambda row: row.CASE_ID if row.parent_is_null == True 
                                         else row.PARNT_CASE_ID, axis = 1)

# Keep cases from 2010 onward
cases2 = cases1[cases1.CASE_FILE_DATE.dt.year >= 2010]

### First merge is between cases and geo_info
We add on PROP_ID column.

In [3]:
m1 = pd.merge(cases2, geo_info1, on = 'CASE_ID', how = 'left', validate = '1:m')

In [4]:
correct_joins = m1[m1.PROP_ID.notna()]
incorrect_joins = m1[m1.PROP_ID.isna()]

In [5]:
print(f"# obs when we join cases and geo_info: {len(m1)}")
print(f"# obs where PROP_ID was NaN: {len(incorrect_joins)}")
print(f"% where PROP_ID was NaN: {len(incorrect_joins) / len(m1)}")

# obs when we join cases and geo_info: 557696
# obs where PROP_ID was NaN: 5781
% where PROP_ID was NaN: 0.010365862405324765


In [6]:
# Of these incorrect joins, do they share parent cases with ones that were joined?
print("# unique parents that were correctly joined, but also appear in incorrect_joins")
print(f"{incorrect_joins[incorrect_joins.PARENT_CASE.isin(correct_joins.PARENT_CASE)].PARENT_CASE.nunique()}")
print(f"# unique parents in incorrect_joins: {incorrect_joins.PARENT_CASE.nunique()}")

# This shows a lot of parent cases won't get joined to a PROP_ID and AIN

# unique parents that were correctly joined, but also appear in incorrect_joins
1709
# unique parents in incorrect_joins: 5471


This is a big rabbit hole that will never get rid of the m:m merge.

1. There are PARENT_CASES where some of their CASE_IDs have PROP_ID merged, and some CASE_IDs that did not correctly join with PROP_ID. These PARENT_IDs will have some obs in correctly_joined and some in incorrectly_joined. This will involve m:m merge.
1. There are also that fall completely within incorrectly_joined, and using PARENT_CASE, we can get PROP_IDs. This will involve m:m merge.
1. There are also PARENT_CASEs that are only in incorrectly_joined, but we cannot get PROP_ID for them at the end of all this.

Since the m:m cannot be averted, let's make it only slightly less painful by linking PROP_ID with AIN by itself, and getting rid of AINs not found in our crosswalk.

### Second merge is between geo_info and la_prop
* To fix these incorrect joins, we would have to have allowed a m:m merge.
* So, let's see if we can circumvent PROP_ID by mapping it to AIN directly.
* But first, we force it to be a 1:m merge, and only keep unique PROP_IDs.
* Then, bring in our crosswalk_parcels_tracts to make sure we only keep parcels we have.

In [7]:
# Do a second merge for PROP_ID and AINs
m2 = (pd.merge(geo_info1[["PROP_ID"]].drop_duplicates(), 
               la_prop1, 
               on = "PROP_ID", how = "left", validate = "1:m")
      .rename(columns = {"ASSR_PRCL_NBR": "AIN"})
     )

m2.head()

,PROP_ID,AIN
0,34237.0,5160003902
1,34306.0,5076007033
2,34323.0,5407005016
3,34169.0,5407002023
4,33937.0,5124001012


In [8]:
crosswalk_parcels_tracts = (pd.read_parquet(
    "s3://city-planning-entitlements/data/crosswalk_parcels_tracts_lacity.parquet")
    [["uuid", "AIN"]]
    )

In [9]:
print(f"# obs in m2: {len(m2)}")
m2 = m2[m2.AIN.isin(crosswalk_parcels_tracts.AIN)]
print(f"# obs in m2 after dropping AINs not in our crosswalk: {len(m2)}")

# obs in m2: 314716
# obs in m2 after dropping AINs not in our crosswalk: 283487


### Third merge is to fix our incorrectly joined df with a m:m merge

* Combine the results of the previous 2 merges. Use PROP_ID to join cases with la_prop.
* Leave the correctly_joined df alone.
* Only do this m:m merge on the incorrectly_joined.

In [10]:
incorrect_joins_with_propid = pd.merge(incorrect_joins.drop(columns = ["PROP_ID"]), 
                                       geo_info1.rename(columns = {"CASE_ID": "PARENT_CASE"}), 
                                       on = "PARENT_CASE", how = "left", validate = "m:m")

print(f"# obs in incorrect_joins before m:m merge: {len(incorrect_joins)}")
print(f"# unqiue PARENT_CASEs in incorrect_joins before m:m merge: {incorrect_joins.PARENT_CASE.nunique()}")
print(f"# obs in incorrect_joins after m:m merge: {len(incorrect_joins_with_propid)}")
print(f"# unqiue PARENT_CASEs in incorrect_joins after m:m merge: {incorrect_joins_with_propid.PARENT_CASE.nunique()}")

# obs in incorrect_joins before m:m merge: 5781
# unqiue PARENT_CASEs in incorrect_joins before m:m merge: 5471
# obs in incorrect_joins after m:m merge: 19809
# unqiue PARENT_CASEs in incorrect_joins after m:m merge: 5471


In [11]:
incorrect_joins_with_ain = pd.merge(incorrect_joins_with_propid, m2,
                                    on = "PROP_ID", how = "left", validate = "m:1"
                                   )

print(f"# obs in incorrect_joins once we add in AIN: {len(incorrect_joins_with_ain)}")
print(f"# unqiue PARENT_CASEs once we add in AIN: {incorrect_joins_with_ain.PARENT_CASE.nunique()}")

# obs in incorrect_joins once we add in AIN: 19809
# unqiue PARENT_CASEs once we add in AIN: 5471


In [12]:
lost_parents = (incorrect_joins_with_ain[incorrect_joins_with_ain.PROP_ID.isna()]
                [["PARENT_CASE"]].drop_duplicates()
               )

print(f"# unique lost PARENT_CASEs: {len(lost_parents)}")
print(f"Double check, try to find some in geo_info: {len(geo_info1[geo_info1.CASE_ID.isin(lost_parents.PARENT_CASE)])}")

# unique lost PARENT_CASEs: 3400
Double check, try to find some in geo_info: 0


In [13]:
# Get rid of obs where we can't link to PROP_ID and AIN
incorrect_joins_now_fixed = incorrect_joins_with_ain[incorrect_joins_with_ain.PROP_ID.notna()]
print(f"# obs in incorrect_joins that were fixed: {len(incorrect_joins_now_fixed)}")
print(f"# unique PARENT_CASEs in incorrect_joins that were fixed: {incorrect_joins_now_fixed.PARENT_CASE.nunique()}")

# obs in incorrect_joins that were fixed: 16307
# unique PARENT_CASEs in incorrect_joins that were fixed: 2071


### Fourth merge is to merge correctly_joined with AIN info
* Once done, can concatenate the correct / incorrect joins together

In [14]:
correct_joins_with_ain = pd.merge(correct_joins, m2, on = "PROP_ID", how = "inner", validate = "m:1")

In [15]:
# Concatenate the 2 parts together
m3 = (pd.concat([
        correct_joins_with_ain, 
        incorrect_joins_now_fixed
    ], axis=0)
      .sort_values(["CASE_ID", "AIN", "PROP_ID"])
      .drop_duplicates(subset = ["CASE_ID", "AIN"])
      .reset_index(drop=True)
)

In [16]:
print(f"# obs in m3: {len(m3)}")
print(f"# obs in m1: {len(m1)}")

print(f"# unique CASE_IDs in m3: {m3.CASE_ID.nunique()}")
print(f"# unique CASE_IDs in m1: {m1.CASE_ID.nunique()}")

print(f"# unique PARENT_CASEs in m3: {m3.PARENT_CASE.nunique()}")
print(f"# unique PARENT_CASEs in m1: {m1.PARENT_CASE.nunique()}")

# obs in m3: 349897
# obs in m1: 557696
# unique CASE_IDs in m3: 49844
# unique CASE_IDs in m1: 54944
# unique PARENT_CASEs in m3: 47046
# unique PARENT_CASEs in m1: 51924


### Fifth merge is to add on project description

In [17]:
m4 = pd.merge(m3, app1, on = "APLC_ID", how = "left", validate = "m:1")

In [18]:
print(f"# obs in m4: {len(m4)}")
print(f"# unique CASE_ID in m4: {m4.CASE_ID.nunique()}")
print(f"# unique PARENT_CASEs in m4: {m4.PARENT_CASE.nunique()}")

# obs in m4: 349897
# unique CASE_ID in m4: 49844
# unique PARENT_CASEs in m4: 47046


In [19]:
m5 = (
    m4.drop(columns = ['PROP_ID', 'parent_is_null'])
    # Nothing dropped here, but just in case
    .drop_duplicates()
    .sort_values(['CASE_ID', 'AIN'])
    .reset_index(drop=True)
)

In [20]:
print(f"# obs in m5: {len(m5)}")
print(f"# unique CASE_ID in m5: {m5.CASE_ID.nunique()}")
print(f"# unique PARENT_CASEs in m5: {m5.PARENT_CASE.nunique()}")

# obs in m5: 349897
# unique CASE_ID in m5: 49844
# unique PARENT_CASEs in m5: 47046
